In [52]:
import math
import numpy as np
import pandas as pd
from numpy.linalg import eig
from numpy.linalg import eigh
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.linear_model import Ridge
import time
from tqdm import tqdm

In [53]:
lookback=252 # training period for factor exposure
numFactors=5
topN=50 # for trading strategy, long stocks with topN exepcted 1-day returns

df=pd.read_table('IJR_20080114.txt')
df['Date']=df['Date'].astype('int')
df.set_index('Date', inplace=True)
df.sort_index(inplace=True)
df.fillna(method='ffill', inplace=True)

In [54]:
dailyret=df.pct_change() # note the rows of dailyret are the observations at different time periods
positionsTable=np.zeros(df.shape)
end_index = df.shape[0]

In [61]:
t_arr = np.arange(lookback+1,end_index)
for t in tqdm(t_arr):
    R=dailyret.iloc[t-lookback+1:t,].T # Dimensions stocks x time
    
    # R.notna().all(axis=1) --> returns array of length = num. stocks and is False if any value for the stock is NA
    # np.where when only a condition is passed selects the indices that contain True
    # Essentially returns indices for stocks whose time series contains all valid daily returns 
    hasData=np.where(R.notna().all(axis=1))[0]
    
    R.dropna(inplace=True) # Drops stocks that contain NA values
    pca = PCA(n_components=numFactors)
    
    # Should we normalize data first?
    # R.T has dimensions time x stocks
    # pca.fit_transform assumes the cols are the number of dimensions; each row then represents a sample 
    # In this case, we have num dimensions = num. stocks 
    # The output projects each sample into lower dimensions, so it has dimensions (num time series vals x num principal components)
    X = pca.fit_transform(R.T)[:,]
    
    # Add a column of ones to X
    # X is now all the training samples projected into lower dimensions with an additional col for 1
    X = sm.add_constant(X) 
    y1 = R.T # Dimensions time x stocks
    
    # MultiOutputRegressor fits one regressor per target (num targets = num stocks in our example)
    # fit_intercept=False sets the y-intercept to 0 (https://stackoverflow.com/a/46781428)
    # This is important since we do not want each target/stock to have its own offset
    # Instead, we want one offset for all targets/stocks, hence the need to add a column of 1's
    clf = MultiOutputRegressor(LinearRegression(fit_intercept=False),n_jobs=4).fit(X, y1)
    
    # Sum of daily returns along all time samples for each stock
    # Is a vector with num elements equal to num. stocks
    Rexp = np.sum(clf.predict(X),axis=0) 
    

    # Index of stock sorted in increasing order
    idxSort=Rexp.argsort()  
    
    # idxSort[:topN] --> return index of stock with topN smallest sum of daily returns
    # idxSort[-topN:-1] --> return index of stock with topN largest sum of daily returns
    # hasData is used since we dropped stocks in dataframe R with NA in time series 
    # The idea is that the factor model predictions have momentum, so they will carry over to the next period
    positionsTable[t, hasData[idxSort[:topN]]]=-1
    positionsTable[t, hasData[idxSort[-topN:-1]]] = 1

100%|██████████| 753/753 [04:38<00:00,  2.70it/s]


In [238]:
# positionsTable is shifted forwards in the time dimension
# We take the absolute value since we only care if the stock has been shorted or longed
# We sum along the stocks dimension, so capital is a vector representing the number of stocks we traded for a given time 
capital=np.nansum(np.array(abs(pd.DataFrame(positionsTable)).shift()), axis=1) 

# For times when no stocks were traded, set position tables to 0 (sanity code)
# positionsTable[capital==0 - 1,]=0 

# Avoids division by zero
# This does not alter the returns since the numerator is guarenteed to be 0 (set by above line)
capital[capital==0]=1

ret=np.nansum(np.array(pd.DataFrame(positionsTable).shift())*np.array(dailyret), axis=1)/capital
avgret=np.nanmean(ret)*252
avgstdev = np.nanstd(ret)*math.sqrt(252)
Sharpe = avgret/avgstdev

print(avgret)
print(avgstdev)
print(Sharpe)

0.029399332677424492
0.06808612790146878
0.43179622022227343
